# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd8c0fb2d90>
├── 'a' --> tensor([[ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030]])
└── 'x' --> <FastTreeValue 0x7fd8c0fb29d0>
    └── 'c' --> tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                        [-0.9420, -0.4518, -0.3240,  1.6024],
                        [ 0.5700, -0.4530,  0.8019,  0.2725]])

In [4]:
t.a

tensor([[ 0.0194, -2.3495, -1.0700],
        [ 0.6895, -0.0388,  2.4030]])

In [5]:
%timeit t.a

58 ns ± 0.0639 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd8c0fb2d90>
├── 'a' --> tensor([[-0.2463,  0.9743, -0.5426],
│                   [ 1.3120,  0.9220, -0.9492]])
└── 'x' --> <FastTreeValue 0x7fd8c0fb29d0>
    └── 'c' --> tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                        [-0.9420, -0.4518, -0.3240,  1.6024],
                        [ 0.5700, -0.4530,  0.8019,  0.2725]])

In [7]:
%timeit t.a = new_value

69.4 ns ± 0.0277 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0194, -2.3495, -1.0700],
               [ 0.6895, -0.0388,  2.4030]]),
    x: Batch(
           c: tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0194, -2.3495, -1.0700],
        [ 0.6895, -0.0388,  2.4030]])

In [11]:
%timeit b.a

55 ns ± 0.0294 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4958,  1.3616, -0.8140],
               [-0.9261,  1.1696, -0.1840]]),
    x: Batch(
           c: tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

876 ns ± 1.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 577 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd818880b80>
├── 'a' --> tensor([[[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]],
│           
│                   [[ 0.0194, -2.3495, -1.0700],
│                    [ 0.6895, -0.0388,  2.4030]]])
└── 'x' --> <FastTreeValue 0x7fd8c0fb2d00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 54.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd811beef40>
├── 'a' --> tensor([[ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030],
│                   [ 0.0194, -2.3495, -1.0700],
│                   [ 0.6895, -0.0388,  2.4030]])
└── 'x' --> <FastTreeValue 0x7fd818a56a60>
    └── 'c' --> tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                        [-0.9420, -0.4518, -0.3240,  1.6024],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 48.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.7 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.3170,  0.3832, -0.7532,  0.4970],
                       [-0.9420, -0.4518, -0.3240,  1.6024],
                       [ 0.5700, -0.4530,  0.8019,  0.2725]],
              
                      [[ 1.3170,  0.3832, -0.7532,  0.4970],
                       [-0.9420, -0.4518, -0.3240,  1.6024],
                       [ 0.5700, -0.4530,  0.8019,  0.2725]],
              
                      [[ 1.3170,  0.3832, -0.7532,  0.4970],
                       [-0.9420, -0.4518, -0.3240,  1.6024],
                       [ 0.5700, -0.4530,  0.8019,  0.2725]],
              
                      [[ 1.3170,  0.3832, -0.7532,  0.4970],
                       [-0.9420, -0.4518, -0.3240,  1.6024],
                       [ 0.5700, -0.4530,  0.8019,  0.2725]],
              
                      [[ 1.3170,  0.3832, -0.7532,  0.4970],
                       [-0.9420, -0.4518, -0.3240,  1.6024],
                       [ 0.5700, -0.4530,  0.8019,  0.2725]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 253 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725],
                      [ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725],
                      [ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725],
                      [ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725],
                      [ 1.3170,  0.3832, -0.7532,  0.4970],
                      [-0.9420, -0.4518, -0.3240,  1.6024],
                      [ 0.5700, -0.4530,  0.8019,  0.2725],
                      [ 1.3170,  0.3832, -0.7532,  0.4970],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 208 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
